# DEL 1
## Byg en statistisk navnegenerator 
I det følgende skal du prøve at bygge en simpel model, der ser på struktur/information i noget så lokalt som bogstavpar, eller rettere: <i>bigrammer</i>.

<b>Inden du kører</b> hver enkelt kode-celle, så prøv først at forstå, hvad koden gør?

In [ ]:
words = open('data/names.txt', 'r').read().splitlines()

### Opgaver:
1. Hvor mange navne er der i datasættet? HINT: <code>len( ... )</code>
2. Print de første 10 navne i datasættet
3. Print det korste - og det længste navn

In [ ]:
b = {} # mapper bigrammer (tuples)
for w in words:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch1, ch2 in zip(chs, chs[1:]):
    bigram = (ch1, ch2)
    b[bigram] = b.get(bigram, 0) + 1

### Hvor ofte forekommer de enkelte bigrammer?

In [ ]:
sorted(b.items(), key = lambda kv: -kv[1])

## PyTorch og tensorer
- Læs på nettet, hvad PyTorch er
- Læs desuden, hvad en <i>tensor</i> er

In [ ]:
import torch

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
print(stoi)

### Hvad er den talmæssige repræsentation af dit eget navn?

In [ ]:
for b in "morten":
    print(f"{b} --> {stoi[b]}")

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)

In [ ]:
xs, ys = [], []

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1
    xs.append(ix1)
    ys.append(ix2)
    
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [ ]:
xs, ys

### Kør koden nedenfor - hvad fortæller plottet?

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

#### Hvad dækker nedenstående over?

In [ ]:
N[0]

#### Hvorfor foretage nedenstående trin?

In [ ]:
p = N[0].float()
p = p / p.sum()
p

#### De bogstaver, vi "plukker" ud her, vil altså optræde med dén sandsynlighed, vi ser ovenfor.

In [ ]:
ix = torch.multinomial(p, num_samples=1, replacement=True).item()
itos[ix]

### Vi laver en 1-dimensionel <code>tensor</code>, <code>p</code>, indlæst med tilfældige tal udtaget fra en ligefordeling (uniform distribution) over intervallet [0, 1). Nedenfor er en smart måde, at "plukke" fra vores lister, tensorer mv. pba. en sandsynlighedsfordeling.

In [ ]:
p = torch.rand(3)
p = p / p.sum()
p

In [ ]:
#torch.multinomial(p, num_samples=100, replacement=True, generator=g)
torch.multinomial(p, num_samples=100, replacement=True)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(N)

In [ ]:
P = N.float()

In [ ]:
P = N.float()
P = P / P.sum(1, keepdim=True)

In [ ]:
P[0]

### Vi kan nu "sample"/generere navne

#### 1.  Baseret på den sandsynlighedsfordeling, vi finder i vores datasæt:

In [ ]:
for i in range(20):
    out = []
    ix = 0
    while True:
        #p = P[ix]
        p = torch.ones(27) / 27.0
        ix = torch.multinomial(p, num_samples=1, replacement=True).item()
        out.append(itos[ix])
        if ix == 0:
            break

    print(''.join(out))

#### 2.  Prøv at ændre <code>p = P[ix]</code> ovenfor til <code>p = torch.ones(27) / 27.0</code>
- Hvad sker der så - og hvad fortæller det os?

### Vi evaluerer modellen: hvad skulle vi forvente? Jo,  1/27 = 0.037 ... 

In [ ]:
log_likelihood = 0.0
n = 0

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    #print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')


print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll}')
print(f'{nll/n}') # gennemsnittet af negative log likelihood

### Overvej sammen med din sidemakker:
- Hvorfor finder modellen "f." som navn?
- Hvordan finder vi parametre, der kan minimere negative log-likelihood?
- Er modellen nogensinde bedre end dataene? Vi kunne jo bruge model-smoothing - hvad er det? Søg på nettet.